In [1]:
import os
import sys

# BIDS Transformers imports
from bids.grabbids import BIDSLayout
from bidst.transformers import ROITransformer
from bidst.transformers import SkullStrippingTransformer

# Nipype example imports
from nipype.interfaces.fsl import ExtractROI
from nipype.interfaces.fsl import BET

# Import for data removal
from shutil import rmtree

# Pipeline import
from sklearn.pipeline import Pipeline


## This is a very basic proof of concept of BIDS Transformers

It implements two BIDS Transformers: one based on FSL ROI extractor and second based on FSL Skull Stripping. Source code can be found at https://github.com/neuro-ml/BIDSTransformers/blob/master/bidst/transformers.py


## To run this notebook you will need the following:

0. Clone this repository (https://github.com/neuro-ml/BIDSTransformers)
1. Working Nipype installation (https://github.com/nipy/nipype, we tested this code on versions 0.12.1 and 0.13.0)
2. PyBids package (https://github.com/INCF/pybids)
3. Download data folder from one of the following links and put it into the folder /bidst/test/data

Link 1: https://drive.google.com/drive/folders/0B6U5KQalulfAOXlHYkdIa25QbG8?usp=sharing

Link 2: https://www.dropbox.com/sh/kh5vkp5s5n6eebh/AABejlvSqHV1HhFy91r9nrT8a?dl=0

## BIDS Transformers Example

In [2]:
#project folder 
project_root = os.path.abspath('bidst/tests/data/ds114')

if os.path.exists(project_root + '/derivatives'):
    rmtree(project_root + '/derivatives')

#### BIDS data structure before tranformations

In [3]:
! tree 'bidst/tests/data/ds114'

bidst/tests/data/ds114
├── dataset_description.json
├── dwi.bval
├── dwi.bvec
├── sub-01
│   ├── ses-retest
│   │   └── dwi
│   │       └── sub-01_ses-retest_dwi.nii.gz
│   └── ses-test
│       └── dwi
│           └── sub-01_ses-test_dwi.nii.gz
├── sub-02
│   ├── ses-retest
│   │   └── dwi
│   │       └── sub-02_ses-retest_dwi.nii.gz
│   └── ses-test
│       └── dwi
│           └── sub-02_ses-test_dwi.nii.gz
├── sub-03
│   ├── ses-retest
│   │   └── dwi
│   │       └── sub-03_ses-retest_dwi.nii.gz
│   └── ses-test
│       └── dwi
│           └── sub-03_ses-test_dwi.nii.gz
├── task-covertverbgeneration_bold.json
├── task-covertverbgeneration_events.tsv
├── task-fingerfootlips_bold.json
├── task-fingerfootlips_events.tsv
├── task-linebisection_bold.json
├── task-overtverbgeneration_bold.json
├── task-overtverbgeneration_events.tsv
├── task-overtwordrepetition_bold.json
└── task-overtwordrepetition_events.tsv

15 directories, 18 files


In [4]:
#users and sessions
user_ids = [('01', 'retest'), ('02', 'retest'), ('03', 'test')]

#ROI extractor transformer
roi = ROITransformer(label='B0',
                     basedir=project_root)

#skull stripping transformer
skull_str = SkullStrippingTransformer(basedir=project_root,
                                      label='B0')
#pipeline definition
pp = Pipeline([('roi', roi),
               ('bet', skull_str)])

#pipeline execution
pp.fit_transform(user_ids)

[('01', 'retest'), ('02', 'retest'), ('03', 'test')]

#### BIDS data structure after tranformations

Transformations were performed only for specified subjects and sessions  

In [5]:
! tree 'bidst/tests/data/ds114'

bidst/tests/data/ds114
├── dataset_description.json
├── derivatives
│   ├── ROITransformer
│   │   ├── ses-retest
│   │   │   └── dwi
│   │   │       ├── sub-01_ses-retest_dwi_label-B0_roi.nii.gz
│   │   │       └── sub-02_ses-retest_dwi_label-B0_roi.nii.gz
│   │   └── ses-test
│   │       └── dwi
│   │           └── sub-03_ses-test_dwi_label-B0_roi.nii.gz
│   └── SkullStrippingTransformer
│       ├── ses-retest
│       │   └── dwi
│       │       ├── sub-01_ses-retest_dwi_label-B0_brain.nii.gz
│       │       └── sub-02_ses-retest_dwi_label-B0_brain.nii.gz
│       └── ses-test
│           └── dwi
│               └── sub-03_ses-test_dwi_label-B0_brain.nii.gz
├── dwi.bval
├── dwi.bvec
├── sub-01
│   ├── ses-retest
│   │   └── dwi
│   │       └── sub-01_ses-retest_dwi.nii.gz
│   └── ses-test
│       └── dwi
│           └── sub-01_ses-test_dwi.nii.gz
├── sub-02
│   ├── ses-retest
│   │   └── dwi
│   │       └── sub-02_ses-retest_dwi.nii.gz
│   └── ses-test
│       └── dwi
│           └── 

# Nipype code for the same thing for comparison

In [8]:
if os.path.exists(project_root + '/derivatives'):
    rmtree(project_root + '/derivatives')

def get_sub_ses_type(file_path):
    dirname = os.path.dirname(file_path)
    sub_ses_type = dirname.split('/')[-3:]
    sub_ses_type = '/'.join(sub_ses_type)
    return sub_ses_type


def get_roi_filename(file_path):
    basename = os.path.basename(file_path)
    no_ext_name = os.path.splitext(basename)[0]
    no_ext_name = os.path.splitext(no_ext_name)[0]
    no_ext_name += '_label-B0_roi'
    normal_name = no_ext_name + '.nii.gz'
    return normal_name


def get_bet_filename(file_path):
    basename = os.path.basename(file_path)
    no_ext_name = os.path.splitext(basename)[0]
    no_ext_name = os.path.splitext(no_ext_name)[0]
    no_ext_name += '_brain'
    normal_name = no_ext_name + '.nii.gz'
    return normal_name    

layout = BIDSLayout(project_root)
files = layout.get(subject='0[12]', session='retest')
file_paths = [f.filename for f in files]
file_paths

['/home/deaddy/Repos/BIDSTransformers/bidst/tests/data/ds114/sub-01/ses-retest/dwi/sub-01_ses-retest_dwi.nii.gz',
 '/home/deaddy/Repos/BIDSTransformers/bidst/tests/data/ds114/sub-02/ses-retest/dwi/sub-02_ses-retest_dwi.nii.gz']

In [9]:
! tree 'bidst/tests/data/ds114'

bidst/tests/data/ds114
├── dataset_description.json
├── dwi.bval
├── dwi.bvec
├── sub-01
│   ├── ses-retest
│   │   └── dwi
│   │       └── sub-01_ses-retest_dwi.nii.gz
│   └── ses-test
│       └── dwi
│           └── sub-01_ses-test_dwi.nii.gz
├── sub-02
│   ├── ses-retest
│   │   └── dwi
│   │       └── sub-02_ses-retest_dwi.nii.gz
│   └── ses-test
│       └── dwi
│           └── sub-02_ses-test_dwi.nii.gz
├── sub-03
│   ├── ses-retest
│   │   └── dwi
│   │       └── sub-03_ses-retest_dwi.nii.gz
│   └── ses-test
│       └── dwi
│           └── sub-03_ses-test_dwi.nii.gz
├── task-covertverbgeneration_bold.json
├── task-covertverbgeneration_events.tsv
├── task-fingerfootlips_bold.json
├── task-fingerfootlips_events.tsv
├── task-linebisection_bold.json
├── task-overtverbgeneration_bold.json
├── task-overtverbgeneration_events.tsv
├── task-overtwordrepetition_bold.json
└── task-overtwordrepetition_events.tsv

15 directories, 18 files


In [10]:
for file_path in file_paths:
    pipeline_name = 'roi_bet'
    sub_ses_type = get_sub_ses_type(file_path)
    roi_path = '{}/derivatives/{}/{}'.format(project_root, pipeline_name, sub_ses_type)
    if not os.path.exists(roi_path):
        os.makedirs(roi_path)
    roi_basename = get_roi_filename(file_path)
    roi_file_path = '{}/{}'.format(roi_path, roi_basename)
    fslroi = ExtractROI(in_file=file_path,
                        roi_file=roi_file_path,
                        t_min=0,
                        t_size=1)
    fslroi.run()
    bet_basename = get_bet_filename(file_path)
    bet_file_path = '{}/{}'.format(roi_path, bet_basename)
    fslbet = BET(in_file=roi_file_path,
                 out_file=bet_file_path)
    fslbet.run()
    print(roi_file_path)
    print(bet_file_path)

/home/deaddy/Repos/BIDSTransformers/bidst/tests/data/ds114/derivatives/roi_bet/sub-01/ses-retest/dwi/sub-01_ses-retest_dwi_label-B0_roi.nii.gz
/home/deaddy/Repos/BIDSTransformers/bidst/tests/data/ds114/derivatives/roi_bet/sub-01/ses-retest/dwi/sub-01_ses-retest_dwi_brain.nii.gz
/home/deaddy/Repos/BIDSTransformers/bidst/tests/data/ds114/derivatives/roi_bet/sub-02/ses-retest/dwi/sub-02_ses-retest_dwi_label-B0_roi.nii.gz
/home/deaddy/Repos/BIDSTransformers/bidst/tests/data/ds114/derivatives/roi_bet/sub-02/ses-retest/dwi/sub-02_ses-retest_dwi_brain.nii.gz


In [11]:
! tree 'bidst/tests/data/ds114'

bidst/tests/data/ds114
├── dataset_description.json
├── derivatives
│   └── roi_bet
│       ├── sub-01
│       │   └── ses-retest
│       │       └── dwi
│       │           ├── sub-01_ses-retest_dwi_brain.nii.gz
│       │           └── sub-01_ses-retest_dwi_label-B0_roi.nii.gz
│       └── sub-02
│           └── ses-retest
│               └── dwi
│                   ├── sub-02_ses-retest_dwi_brain.nii.gz
│                   └── sub-02_ses-retest_dwi_label-B0_roi.nii.gz
├── dwi.bval
├── dwi.bvec
├── sub-01
│   ├── ses-retest
│   │   └── dwi
│   │       └── sub-01_ses-retest_dwi.nii.gz
│   └── ses-test
│       └── dwi
│           └── sub-01_ses-test_dwi.nii.gz
├── sub-02
│   ├── ses-retest
│   │   └── dwi
│   │       └── sub-02_ses-retest_dwi.nii.gz
│   └── ses-test
│       └── dwi
│           └── sub-02_ses-test_dwi.nii.gz
├── sub-03
│   ├── ses-retest
│   │   └── dwi
│   │       └── sub-03_ses-retest_dwi.nii.gz
│   └── ses-test
│       └── dwi
│           └── sub-03_ses-test_dwi.nii.gz